In [1]:
import random
import matplotlib
import time
from IPython.display import clear_output

In [2]:
%matplotlib inline

### Dictionaries

In [3]:
# dictionary: continent name and all territory_id within continent
continent_id={
    "North America":  [1, 2, 3, 4, 5, 6, 7, 8, 9],
    "South America": [10, 11, 12, 13],
    "Europe": [14, 15, 16, 17, 18, 19, 20],
    "Africa": [21, 22, 23, 24, 25, 26],
    "Asia": [27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38],
    "Australia": [39, 40, 41, 42]    
}

# dictionary: territory_id and its name
territory_id={
    1: "Alaska", 2: "Alberta", 3: "Central America", 4: "Eastern United States", 5: "Greenland",
    6: "Northwest Territory", 7: "Ontario", 8: "Quebec", 9: "Western United States",10: "Argentina", 
    11: "Brazil", 12: "Peru", 13: "Venezuela", 14: "Great Britain", 15: "Iceland", 
    16: "Northern Europe", 17: "Scandinavia", 18: "Southern Europe", 19: "Ukraine", 20: "Western Europe",
    21: "Congo", 22: "East Africa", 23: "Egypt", 24: "Madagascar", 25: "North Africa",
    26: "South Africa", 27: "Afghanistan", 28: "China", 29: "India", 30: "Irkutsk",
    31: "Japan", 32: "Kamchatka", 33: "Middle East", 34: "Mongolia", 35: "Siam ",
    36: "Siberia", 37: "Ural", 38: "Yakutsk", 39: "Eastern Australia", 40: "Indonesia",
    41: "New Guinea",42: "Western Australia"
}

# dictionary: territory_id and all territory_id it borders with
neighbour_id={
    1: [2, 6, 32],
    2: [1, 6, 7, 9],
    3: [4, 9, 13],
    4: [3, 7, 8, 9],
    5: [6, 7, 8, 15],
    6: [1, 2, 5, 7],
    7: [2, 4, 5, 6, 8, 9],
    8: [4, 5, 7],
    9: [2, 3, 4, 7],
    10: [11, 12],
    11: [10, 12, 13, 25],
    12: [10, 11, 13],
    13: [3, 11, 12],
    14: [15, 16, 17, 20],
    15: [5, 14, 17],
    16: [14, 17, 18, 19, 20],
    17: [14, 15, 16, 19],
    18: [16, 19, 20, 23, 25, 33],
    19: [16, 17, 27, 33, 37],
    20: [14, 16, 18, 25],
    21: [22, 25, 26],
    22: [21, 23, 24, 25, 26, 33],
    23: [18, 22, 25, 33],
    24: [22, 26],
    25: [11, 18, 20, 21, 22, 23],
    26: [21, 22, 24],
    27: [19, 28, 29, 33, 37],
    28: [27, 29, 34, 35, 36, 37],
    29: [27, 28, 33, 35],
    30: [32, 34, 36, 38],
    31: [32, 34],
    32: [1, 30, 31, 34, 38],
    33: [18, 19, 22, 23, 27, 29],
    34: [28, 30, 31, 32, 36],
    35: [28, 29, 40],
    36: [28, 30, 34, 37, 38],
    37: [19, 27, 28, 36],
    38: [30, 32, 36],
    39: [41, 42],
    40: [35, 41, 42],
    41: [39, 40, 42],
    42: [39, 40, 41]
}

In [4]:
# dictionary: player_id and a list of its colour
colour_id={
    1: "red",
    2: "blue",
    3: "green",
    4: "orange",
    5: "purple",
    6: "yellow",
    "": "black"
}

In [5]:
# dictionary: territory_id and a list of its owners id with the number of troops occupied with
territory_occupation={
    1: [1, 4],
    2: [2, 1],
    3: [1, 1],
    4: [1, 1],
    5: ["", 0],
    6: [1, 1],
    7: ["", 0],
    8: ["", 0],
    9: ["", 0],
    10: ["", 0],
    11: ["", 0],
    12: ["", 0],
    13: ["", 0],
    14: ["", 0],
    15: ["", 0],
    16: ["", 0],
    17: ["", 0],
    18: ["", 0],
    19: ["", 0],
    20: ["", 0],
    21: ["", 0],
    22: ["", 0],
    23: ["", 0],
    24: ["", 0],
    25: ["", 0],
    26: ["", 0],
    27: ["", 0],
    28: ["", 0],
    29: ["", 0],
    30: ["", 0],
    31: ["", 0],
    32: ["", 0],
    33: ["", 0],
    34: ["", 0],
    35: ["", 0],
    36: ["", 0],
    37: ["", 0],
    38: ["", 0],
    39: ["", 0],
    40: ["", 0],
    41: ["", 0],
    42: ["", 0],
}

## Player class

In [6]:
class Player:

    def __init__(self,id_player):
        self.name='' #name_player
        self.id=id_player  # position of the player in the game
        self.occupied_territories=dict() # dictionnary with the territories owned and the number of troops on each territory
        self.riskcards=[]
        self.missions=[] #Later maybe
        self.stock=0 # counter for the reinforcement phase
        
        
     #provide the list of territories occupied by the player 
    def get_occupied_territories(self):
        self.occupied_territories={x:territory_occupation[x][1] for x in territory_occupation.keys() if territory_occupation[x][0]==self.id}
        return self.occupied_territories
    
    def move(self, from_territory_id, to_territory_id, troops):
        '''
        Validity checks on:
        1) from_territory belongs to player
        2) selected troops are available (i.e. enough troops at from_territory and at least 1 is remain stationed)
        3) to_territory is a neighbour of from_territory
        4) to_territory owner is not player
        '''
        outcome = (from_territory_id in self.occupied_territories.keys(), territory_occupation[from_territory_id][1]-troops>=1, to_territory_id in neighbour_id[from_territory_id], to_territory_id not in self.occupied_territories.keys())
        return outcome
    
    def attack_m(self, from_territory_id, to_territory_id, troops):
        # Mechanism for an attack
        
        # Valid move check
        if sum(self.move(from_territory_id, to_territory_id, troops))==4:
            original_force=troops
            # Update territory_occupation dictionary to move troops out of from_territory
            territory_occupation[from_territory_id][1]-=original_force
            # loop until "last-man standing"
            while (troops>0) & (territory_occupation[to_territory_id][1]>0):
                attack, defence = self.roll_dice()
                print(f"Attacker: {troops} vs Defender: {territory_occupation[to_territory_id][1]}")
                time.sleep(1)
                print(f"Attacker: {attack} vs Defender: {defence}")
                time.sleep(1)
                if sum(attack)==sum(defence):
                    print(f"It's a draw, another battle begins.")
                elif sum(attack)>sum(defence):
                    print(f"Attacker kills one troop of the defender.")
                    territory_occupation[to_territory_id][1]-=1               
                else:
                    print(f"Attacker lost one troop.")
                    troops-=1
                time.sleep(1)
                
            # check if attempted conquer was a success    
            if troops>0:
                self.get_occupied_territories()
                territory_occupation[to_territory_id]=[self.id, troops]
                print(f"Successfully conquered {territory_id[to_territory_id]}, moving in with {troops}", "troops." if troops>1 else "troop.")
                
                if self.win_check():
                    print("You have won the game.")
                else:
                    pass
            
            else:
                print(f"Failed attempt to conquer {territory_id[to_territory_id]}, {original_force}", "troops were" if original_force>1 else "troop was", "killed.")
            self.get_occupied_territories() 
        else:
            print("criteria not met for an attack")
    
    def attack(self):
        # Update player's occupied_territories dictionary
        self.get_occupied_territories()
        attack_on = True
        while attack_on:
            print(f"""Current territories:
{[territory_id[i] for i,j in self.occupied_territories.items()]}""")
            if input('Would you like to make any attacks? Y/N ').upper() == "Y":
                clear_output()
                attack_input=True
                while attack_input:
                    try:
                        print(f"""Current territories:
{[territory_id[i] for i,j in self.occupied_territories.items()]}""")
                        self.attack_m(self.convert_territory_id(input(f"""Please enter the territory you would want to attack from: """).title()), self.convert_territory_id(input('Please enter the territory you would want to attack: ').title()), int(input('Enter the number of troops to deploy for this attack: ')))
                    except:
                        print("Please make sure you have enter the information in the correct format.")
                    else:
                        break
            else:
                attack_on=False

    def roll_dice(self):
        red_dice, blue_dice=random.choices(range(1,7), k=3), random.choices(range(1,7), k=3)
        return red_dice, blue_dice

    def manoeuvre_m(self, from_territory_id, to_territory_id, troops):
        # Mechanism for a manoeuvre
        
        # Valid move check
        if sum(self.move(from_territory_id, to_territory_id, troops)[:3])==3 & sum(self.move(from_territory_id, to_territory_id, troops))!=4:
            territory_occupation[to_territory_id][1]+=troops
            territory_occupation[from_territory_id][1]-=troops
            print(f"You have moved {troops}",  "troops" if troops>1 else "troop", f"into {territory_id[to_territory_id]}.")
            self.get_occupied_territories()
        else:
            print("criteria not met for a manoeuvre")
    
    def manoeuvre(self):
        # Phase 3: Manoeuvre, ask obtain player input
        # Update player's occupied territories dictionary
        self.get_occupied_territories()
        manoeuvre_on = True
        while manoeuvre_on:
            print(f"""Current territories:
{[territory_id[i] for i,j in self.occupied_territories.items()]}""")
            if input('Would you like to make any manoeuvre? Y/N ').upper() == "Y":
                clear_output()
                manoeuvre_input=True
                while manoeuvre_input:
                    try:
                        print(f"""Current territories:
{[territory_id[i] for i,j in self.occupied_territories.items()]}""")
                        self.manoeuvre_m(self.convert_territory_id(input(f"""Please enter the territory you would want to manoeuvre from: """).title()), self.convert_territory_id(input('Please enter the territory you would want to manoeuvre to: ').title()), int(input('Enter the number of troops to deploy for this manoeuvre: ')))
                    except:
                        print("Please make sure you have enter the information in the correct format.")
                    else:
                        break
            else:
                manoeuvre_on=False
    
    def win_check(self):
        return len(self.occupied_territories)>=30
        pass
    
    def continent_check(self):
        # returns a dictionary of continent and True/False on whether player has conquered the whole continent
        progress={i: set(j).issubset(self.occupied_territories.keys()) for i,j in continent_id.items()}
        return progress
    
    def convert_territory_id(self, territory):
        return [i for i,j in territory_id.items() if j==territory][0]
        
    

In [ ]:
#for 2 players / to be adapted for more player
Nb_player=2
#choose of Number of troops in each game depending on the number of players
def game_setup(NbPlayer):
    army_size={2:40,3:35,4:30,5:35,6:20}
    return army_size[NbPlayer]

def assign_territories(Nb_player):
    #shuffle the territories
    import random
    deck_of_territories=random.sample(territory_id.keys(),len(territory_id.keys()))

    #distribute all the territories randomly to the players and add exactly one troop of the player  on the territory.

    for i,territory in enumerate(deck_of_territories):
        player=Joueur1
        if i%2==0:
            player=Joueur2   
        player.occupied_territories[territory]=1
        #Update the territory_occupation
        territory_occupation[i]=[player.id,1]
#Update stock of remaining Troops for each player
Joueur1.stock=game_setup(Nb_player)-len(list(Joueur1.occupied_territories.keys()))
Joueur2.stock=game_setup(Nb_player)-len(list(Joueur2.occupied_territories.keys())) 

print(Joueur2.stock)
print(Joueur1.stock)
print(Joueur2.occupied_territories.keys())
print(Joueur1.occupied_territories.keys())

In [ ]:
game_setup(2)

# Card Dictionary
card_dict=territory_id.copy()
card_dict[0]="Wild Card"
card_dict[43]="Wild Card"

In [7]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

In [ ]:
test="<span style='color:red'>"
test1="testing"
colortest=colour_id[territory_occupation[1][0]]

&nbsp;

In [8]:
def test():
    printmd(f"<span style='color:{colour_id[territory_occupation[1][0]]}'>{territory_occupation[1][1]}</span>")
    return

In [76]:
def board():
    printmd(f"""&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[1][0]]}'>aa</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[6][0]]}'>bbbb&nbsp;&nbsp;b</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[5][0]]}'>cccccc</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[17][0]]}'>pp</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[19][0]]}'>t</span><span style='color:{colour_id[territory_occupation[37][0]]}'>B&nbsp;&nbsp;B&nbsp;&nbsp;B</span><span style='color:{colour_id[territory_occupation[36][0]]}'>CCCCC</span><span style='color:{colour_id[territory_occupation[38][0]]}'>DDDDDDDD</span><span style='color:{colour_id[territory_occupation[32][0]]}'>FFFF&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span>
&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[1][0]]}'>aaaaaaa</span><span style='color:{colour_id[territory_occupation[6][0]]}'>bbbbbbbbbb</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[5][0]]}'>cccc</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[17][0]]}'>ppp</span><span style='color:{colour_id[territory_occupation[19][0]]}'>tt&nbsp;&nbsp;ttt</span><span style='color:{colour_id[territory_occupation[37][0]]}'>BBBBB</span><span style='color:{colour_id[territory_occupation[36][0]]}'>CCCCC</span><span style='color:{colour_id[territory_occupation[38][0]]}'>DDDDDDD</span><span style='color:{colour_id[territory_occupation[32][0]]}'>FFFFFFFFFFF&nbsp;&nbsp;</span>
&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[1][0]]}'>aaaaaaa</span><span style='color:{colour_id[territory_occupation[6][0]]}'>bbbbbbbbbbb</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[5][0]]}'>ccc</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[15][0]]}'>nnn</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[17][0]]}'>pp&nbsp;&nbsp;p</span><span style='color:{colour_id[territory_occupation[19][0]]}'>tttttt</span><span style='color:{colour_id[territory_occupation[37][0]]}'>BBBBB</span><span style='color:{colour_id[territory_occupation[36][0]]}'>CCCCC</span><span style='color:{colour_id[territory_occupation[38][0]]}'>DDDDD</span><span style='color:{colour_id[territory_occupation[32][0]]}'>FFFFFFFFFFFF&nbsp;&nbsp;F</span>
&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[1][0]]}'>aaaaaaaa</span><span style='color:{colour_id[territory_occupation[2][0]]}'>ddddddd</span><span style='color:{colour_id[territory_occupation[7][0]]}'>e</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[8][0]]}'>fff</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[5][0]]}'>c</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[14][0]]}'>o</span>&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[17][0]]}'>pp</span>&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[19][0]]}'>ttttttt</span><span style='color:{colour_id[territory_occupation[37][0]]}'>BBBBB</span><span style='color:{colour_id[territory_occupation[36][0]]}'>CCC</span><span style='color:{colour_id[territory_occupation[30][0]]}'>EEEEEEE</span><span style='color:{colour_id[territory_occupation[32][0]]}'>FFFFFFFFFF</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[1][0]]}'>a&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;a</span><span style='color:{colour_id[territory_occupation[2][0]]}'>ddddddd</span><span style='color:{colour_id[territory_occupation[7][0]]}'>eee</span>&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[8][0]]}'>fff</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[14][0]]}'>oo</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[17][0]]}'>p</span>&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[16][0]]}'>r</span><span style='color:{colour_id[territory_occupation[19][0]]}'>ttttttt</span><span style='color:{colour_id[territory_occupation[37][0]]}'>BBBBB</span><span style='color:{colour_id[territory_occupation[36][0]]}'>CC</span><span style='color:{colour_id[territory_occupation[30][0]]}'>EEEEEEEE</span><span style='color:{colour_id[territory_occupation[32][0]]}'>FFF&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;F</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[1][0]]}'>a</span><span style='color:{colour_id[territory_occupation[2][0]]}'>ddddddd</span><span style='color:{colour_id[territory_occupation[7][0]]}'>eeee</span><span style='color:{colour_id[territory_occupation[8][0]]}'>fffff</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[14][0]]}'>ooo</span>&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[16][0]]}'>rrrr</span><span style='color:{colour_id[territory_occupation[19][0]]}'>ttttt</span><span style='color:{colour_id[territory_occupation[27][0]]}'>GGGG</span><span style='color:{colour_id[territory_occupation[37][0]]}'>BBB</span><span style='color:{colour_id[territory_occupation[36][0]]}'>CC</span><span style='color:{colour_id[territory_occupation[30][0]]}'>EEEE</span><span style='color:{colour_id[territory_occupation[34][0]]}'>HHHHH</span><span style='color:{colour_id[territory_occupation[32][0]]}'>FFF</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[2][0]]}'>dddddd</span><span style='color:{colour_id[territory_occupation[7][0]]}'>eeee<span><span style='color:{colour_id[territory_occupation[8][0]]}'>ffff&nbsp;&nbsp;f</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[20][0]]}'>qq</span><span style='color:{colour_id[territory_occupation[16][0]]}'>rrrr</span><span style='color:{colour_id[territory_occupation[19][0]]}'>ttttt</span><span style='color:{colour_id[territory_occupation[27][0]]}'>GGGGGG</span><span style='color:{colour_id[territory_occupation[37][0]]}'>B</span><span style='color:{colour_id[territory_occupation[28][0]]}'>IIII</span><span style='color:{colour_id[territory_occupation[34][0]]}'>HHHHHHHHH&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[9][0]]}'>ggggggg</span><span style='color:{colour_id[territory_occupation[4][0]]}'>hh</span>&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[8][0]]}'>ffff</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[20][0]]}'>qq</span><span style='color:{colour_id[territory_occupation[18][0]]}'>sssss</span>&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[19][0]]}'>ttt</span>&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[27][0]]}'>GGGGGG</span><span style='color:{colour_id[territory_occupation[28][0]]}'>IIIII</span><span style='color:{colour_id[territory_occupation[34][0]]}'>HHHHHHHH&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[9][0]]}'>ggggggg</span><span style='color:{colour_id[territory_occupation[4][0]]}'>hhhhh</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[20][0]]}'>qqq</span>&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[18][0]]}'>ss&nbsp;&nbsp;ss</span>&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[19][0]]}'>tt</span>&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[27][0]]}'>GGGGGG</span><span style='color:{colour_id[territory_occupation[28][0]]}'>IIIIIIIIIII</span>&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[31][0]]}'>J&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[9][0]]}'>gggggg</span><span style='color:{colour_id[territory_occupation[4][0]]}'>hhhh</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[20][0]]}'>qq</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[18][0]]}'>s&nbsp;&nbsp;ss</span><span style='color:{colour_id[territory_occupation[33][0]]}'>AAAAAA</span><span style='color:{colour_id[territory_occupation[29][0]]}'>KKK</span><span style='color:{colour_id[territory_occupation[27][0]]}'>GG</span><span style='color:{colour_id[territory_occupation[28][0]]}'>IIIIIIIII&nbsp;&nbsp;I</span>&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[31][0]]}'>JJ&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[9][0]]}'>gggg</span><span style='color:{colour_id[territory_occupation[4][0]]}'>hhhhhh</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[25][0]]}'>uuuu</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[33][0]]}'>AAAAA</span><span style='color:{colour_id[territory_occupation[29][0]]}'>KKKKKK</span><span style='color:{colour_id[territory_occupation[28][0]]}'>IIIIIIII</span>&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[31][0]]}'>JJ&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[9][0]]}'>ggg</span><span style='color:{colour_id[territory_occupation[4][0]]}'>hhh&nbsp;&nbsp;h</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[25][0]]}'>uuuuuu</span><span style='color:{colour_id[territory_occupation[23][0]]}'>vvvv</span><span style='color:{colour_id[territory_occupation[33][0]]}'>AAA&nbsp;&nbsp;A</span><span style='color:{colour_id[territory_occupation[29][0]]}'>KKKKKKK</span><span style='color:{colour_id[territory_occupation[28][0]]}'>IIIIIII</span>&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[31][0]]}'>J&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[3][0]]}'>ii</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[25][0]]}'>uuuuuuu</span><span style='color:{colour_id[territory_occupation[23][0]]}'>vvvv</span>&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[33][0]]}'>AAAA</span>&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[29][0]]}'>KKKKKK</span><span style='color:{colour_id[territory_occupation[35][0]]}'>LLLL</span><span style='color:{colour_id[territory_occupation[28][0]]}'>II&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[3][0]]}'>ii</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[25][0]]}'>uuuuuuu</span><span style='color:{colour_id[territory_occupation[23][0]]}'>vvvvv</span>&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[33][0]]}'>AAA</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[29][0]]}'>KKKK</span>&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[35][0]]}'>LLLL&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[3][0]]}'>iii</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[25][0]]}'>uuuuuuuu</span><span style='color:{colour_id[territory_occupation[22][0]]}'>wwww</span>&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[33][0]]}'>AA</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[29][0]]}'>KK</span>&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[35][0]]}'>LLL&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[3][0]]}'>iii</span><span style='color:{colour_id[territory_occupation[13][0]]}'>jj</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[25][0]]}'>uuuuuuuu</span><span style='color:{colour_id[territory_occupation[22][0]]}'>wwwww</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[29][0]]}'>K</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[35][0]]}'>L</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[40][0]]}'>M</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[13][0]]}'>jjjjj</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[25][0]]}'>uuuuuu</span><span style='color:{colour_id[territory_occupation[21][0]]}'>xx</span><span style='color:{colour_id[territory_occupation[22][0]]}'>wwwww</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[29][0]]}'>K</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[40][0]]}'>M&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;MM</span>&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[41][0]]}'>NN</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[12][0]]}'>k</span><span style='color:{colour_id[territory_occupation[13][0]]}'>jjj</span><span style='color:{colour_id[territory_occupation[11][0]]}'>mmmm</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[25][0]]}'>u</span><span style='color:{colour_id[territory_occupation[21][0]]}'>xx</span><span style='color:{colour_id[territory_occupation[22][0]]}'>www</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[40][0]]}'>MMMM</span>&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[41][0]]}'>NNNN&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[12][0]]}'>kk</span><span style='color:{colour_id[territory_occupation[11][0]]}'>mmmmmmmm</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[21][0]]}'>xxx</span><span style='color:{colour_id[territory_occupation[22][0]]}'>www</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[41][0]]}'>N&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[12][0]]}'>kkk</span><span style='color:{colour_id[territory_occupation[11][0]]}'>mmmmm</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[21][0]]}'>xxx</span><span style='color:{colour_id[territory_occupation[26][0]]}'>yyy</span>&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[24][0]]}'>zz</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[39][0]]}'>PPPP&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[10][0]]}'>l</span><span style='color:{colour_id[territory_occupation[12][0]]}'>kk</span><span style='color:{colour_id[territory_occupation[11][0]]}'>mmmm</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[26][0]]}'>yyyyy</span>&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[24][0]]}'>z</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[42][0]]}'>OO</span><span style='color:{colour_id[territory_occupation[39][0]]}'>PPPPP&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[10][0]]}'>lllll</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[26][0]]}'>yyyy</span>&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[24][0]]}'>z</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[42][0]]}'>OOOOO</span><span style='color:{colour_id[territory_occupation[39][0]]}'>PPPP&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[10][0]]}'>llll</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[26][0]]}'>yyy</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[42][0]]}'>OOOO</span><span style='color:{colour_id[territory_occupation[39][0]]}'>PPPP&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[10][0]]}'>lll</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[26][0]]}'>yy</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[42][0]]}'>OO</span>&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[39][0]]}'>PPPP&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[10][0]]}'>l&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span>

""")
    return

In [77]:
board()

&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:red'>aa</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:red'>bbbb&nbsp;&nbsp;b</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>cccccc</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>pp</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>t</span><span style='color:black'>B&nbsp;&nbsp;B&nbsp;&nbsp;B</span><span style='color:black'>CCCCC</span><span style='color:black'>DDDDDDDD</span><span style='color:black'>FFFF&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span>
&nbsp;&nbsp;<span style='color:red'>aaaaaaa</span><span style='color:red'>bbbbbbbbbb</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>cccc</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>ppp</span><span style='color:black'>tt&nbsp;&nbsp;ttt</span><span style='color:black'>BBBBB</span><span style='color:black'>CCCCC</span><span style='color:black'>DDDDDDD</span><span style='color:black'>FFFFFFFFFFF&nbsp;&nbsp;</span>
&nbsp;&nbsp;<span style='color:red'>aaaaaaa</span><span style='color:red'>bbbbbbbbbbb</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>ccc</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>nnn</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>pp&nbsp;&nbsp;p</span><span style='color:black'>tttttt</span><span style='color:black'>BBBBB</span><span style='color:black'>CCCCC</span><span style='color:black'>DDDDD</span><span style='color:black'>FFFFFFFFFFFF&nbsp;&nbsp;F</span>
&nbsp;&nbsp;<span style='color:red'>aaaaaaaa</span><span style='color:blue'>ddddddd</span><span style='color:black'>e</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>fff</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>c</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>o</span>&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>pp</span>&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>ttttttt</span><span style='color:black'>BBBBB</span><span style='color:black'>CCC</span><span style='color:black'>EEEEEEE</span><span style='color:black'>FFFFFFFFFF</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:red'>a&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;a</span><span style='color:blue'>ddddddd</span><span style='color:black'>eee</span>&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>fff</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>oo</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>p</span>&nbsp;&nbsp;<span style='color:black'>r</span><span style='color:black'>ttttttt</span><span style='color:black'>BBBBB</span><span style='color:black'>CC</span><span style='color:black'>EEEEEEEE</span><span style='color:black'>FFF&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;F</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:red'>a</span><span style='color:blue'>ddddddd</span><span style='color:black'>eeee</span><span style='color:black'>fffff</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>ooo</span>&nbsp;&nbsp;<span style='color:black'>rrrr</span><span style='color:black'>ttttt</span><span style='color:black'>GGGG</span><span style='color:black'>BBB</span><span style='color:black'>CC</span><span style='color:black'>EEEE</span><span style='color:black'>HHHHH</span><span style='color:black'>FFF</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:blue'>dddddd</span><span style='color:black'>eeee<span><span style='color:black'>ffff&nbsp;&nbsp;f</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>qq</span><span style='color:black'>rrrr</span><span style='color:black'>ttttt</span><span style='color:black'>GGGGGG</span><span style='color:black'>B</span><span style='color:black'>IIII</span><span style='color:black'>HHHHHHHHH&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>ggggggg</span><span style='color:red'>hh</span>&nbsp;&nbsp;<span style='color:black'>ffff</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>qq</span><span style='color:black'>sssss</span>&nbsp;&nbsp;<span style='color:black'>ttt</span>&nbsp;&nbsp;<span style='color:black'>GGGGGG</span><span style='color:black'>IIIII</span><span style='color:black'>HHHHHHHH&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>ggggggg</span><span style='color:red'>hhhhh</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>qqq</span>&nbsp;&nbsp;<span style='color:black'>ss&nbsp;&nbsp;ss</span>&nbsp;&nbsp;<span style='color:black'>tt</span>&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>GGGGGG</span><span style='color:black'>IIIIIIIIIII</span>&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>J&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>gggggg</span><span style='color:red'>hhhh</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>qq</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>s&nbsp;&nbsp;ss</span><span style='color:black'>AAAAAA</span><span style='color:black'>KKK</span><span style='color:black'>GG</span><span style='color:black'>IIIIIIIII&nbsp;&nbsp;I</span>&nbsp;&nbsp;<span style='color:black'>JJ&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>gggg</span><span style='color:red'>hhhhhh</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>uuuu</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>AAAAA</span><span style='color:black'>KKKKKK</span><span style='color:black'>IIIIIIII</span>&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>JJ&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>ggg</span><span style='color:red'>hhh&nbsp;&nbsp;h</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>uuuuuu</span><span style='color:black'>vvvv</span><span style='color:black'>AAA&nbsp;&nbsp;A</span><span style='color:black'>KKKKKKK</span><span style='color:black'>IIIIIII</span>&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>J&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:red'>ii</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>uuuuuuu</span><span style='color:black'>vvvv</span>&nbsp;&nbsp;<span style='color:black'>AAAA</span>&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>KKKKKK</span><span style='color:black'>LLLL</span><span style='color:black'>II&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:red'>ii</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>uuuuuuu</span><span style='color:black'>vvvvv</span>&nbsp;&nbsp;<span style='color:black'>AAA</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>KKKK</span>&nbsp;&nbsp;<span style='color:black'>LLLL&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:red'>iii</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>uuuuuuuu</span><span style='color:black'>wwww</span>&nbsp;&nbsp;<span style='color:black'>AA</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>KK</span>&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>LLL&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:red'>iii</span><span style='color:black'>jj</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>uuuuuuuu</span><span style='color:black'>wwwww</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>K</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>L</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>M</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>jjjjj</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>uuuuuu</span><span style='color:black'>xx</span><span style='color:black'>wwwww</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>K</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>M&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;MM</span>&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>NN</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>k</span><span style='color:black'>jjj</span><span style='color:black'>mmmm</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>u</span><span style='color:black'>xx</span><span style='color:black'>www</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>MMMM</span>&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>NNNN&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>kk</span><span style='color:black'>mmmmmmmm</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>xxx</span><span style='color:black'>www</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>N&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>kkk</span><span style='color:black'>mmmmm</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>xxx</span><span style='color:black'>yyy</span>&nbsp;&nbsp;<span style='color:black'>zz</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>PPPP&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>l</span><span style='color:black'>kk</span><span style='color:black'>mmmm</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>yyyyy</span>&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>z</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>OO</span><span style='color:black'>PPPPP&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>lllll</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>yyyy</span>&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>z</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>OOOOO</span><span style='color:black'>PPPP&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>llll</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>yyy</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>OOOO</span><span style='color:black'>PPPP&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>lll</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>yy</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>OO</span>&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>PPPP&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>l&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span>



In [75]:
printmd(f"""
&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[1][0]]}'>aaa</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[6][0]]}'>bbbbb&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;b</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[5][0]]}'>cccccc</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[17][0]]}'>pp</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[19][0]]}'>t</span><span style='color:{colour_id[territory_occupation[37][0]]}'>B&nbsp;&nbsp;B&nbsp;&nbsp;B</span><span style='color:{colour_id[territory_occupation[36][0]]}'>CCCCC</span><span style='color:{colour_id[territory_occupation[38][0]]}'>DDDDDDDD</span><span style='color:{colour_id[territory_occupation[32][0]]}'>FFFF&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span>
&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[1][0]]}'>aaaaaaaa</span><span style='color:{colour_id[territory_occupation[6][0]]}'>bbbbbbbbbbbb</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[5][0]]}'>cccc</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[17][0]]}'>ppp</span><span style='color:{colour_id[territory_occupation[19][0]]}'>tt&nbsp;&nbsp;ttt</span><span style='color:{colour_id[territory_occupation[37][0]]}'>BBBBB</span><span style='color:{colour_id[territory_occupation[36][0]]}'>CCCCC</span><span style='color:{colour_id[territory_occupation[38][0]]}'>DDDDDDD</span><span style='color:{colour_id[territory_occupation[32][0]]}'>FFFFFFFFFFF&nbsp;&nbsp;</span>
&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[1][0]]}'>aaaaaaaa</span><span style='color:{colour_id[territory_occupation[6][0]]}'>bbbbbbbbbbbbb</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[5][0]]}'>ccc</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[15][0]]}'>nnn</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[17][0]]}'>pp&nbsp;&nbsp;p</span><span style='color:{colour_id[territory_occupation[19][0]]}'>tttttt</span><span style='color:{colour_id[territory_occupation[37][0]]}'>BBBBB</span><span style='color:{colour_id[territory_occupation[36][0]]}'>CCCCC</span><span style='color:{colour_id[territory_occupation[38][0]]}'>DDDDD</span><span style='color:{colour_id[territory_occupation[32][0]]}'>FFFFFFFFFFFF&nbsp;&nbsp;F</span>
&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[1][0]]}'>aaaaaaaaa</span><span style='color:{colour_id[territory_occupation[2][0]]}'>dddddddd</span><span style='color:{colour_id[territory_occupation[7][0]]}'>ee</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[8][0]]}'>fff</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[5][0]]}'>c</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[14][0]]}'>o</span>&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[17][0]]}'>pp</span>&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[19][0]]}'>ttttttt</span><span style='color:{colour_id[territory_occupation[37][0]]}'>BBBBB</span><span style='color:{colour_id[territory_occupation[36][0]]}'>CCC</span><span style='color:{colour_id[territory_occupation[30][0]]}'>EEEEEEE</span><span style='color:{colour_id[territory_occupation[32][0]]}'>FFFFFFFFFF</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[1][0]]}'>a&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;a</span><span style='color:{colour_id[territory_occupation[2][0]]}'>dddddddd</span><span style='color:{colour_id[territory_occupation[7][0]]}'>eeee</span>&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[8][0]]}'>ffff</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[14][0]]}'>oo</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[17][0]]}'>p</span>&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[16][0]]}'>r</span><span style='color:{colour_id[territory_occupation[19][0]]}'>ttttttt</span><span style='color:{colour_id[territory_occupation[37][0]]}'>BBBBB</span><span style='color:{colour_id[territory_occupation[36][0]]}'>CC</span><span style='color:{colour_id[territory_occupation[30][0]]}'>EEEEEEEE</span><span style='color:{colour_id[territory_occupation[32][0]]}'>FFF&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;F</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[1][0]]}'>a</span><span style='color:{colour_id[territory_occupation[2][0]]}'>dddddddd</span><span style='color:{colour_id[territory_occupation[7][0]]}'>eeeee</span><span style='color:{colour_id[territory_occupation[8][0]]}'>ffffff</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[14][0]]}'>ooo</span>&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[16][0]]}'>rrrr</span><span style='color:{colour_id[territory_occupation[19][0]]}'>ttttt</span><span style='color:{colour_id[territory_occupation[27][0]]}'>GGGG</span><span style='color:{colour_id[territory_occupation[37][0]]}'>BBB</span><span style='color:{colour_id[territory_occupation[36][0]]}'>CC</span><span style='color:{colour_id[territory_occupation[30][0]]}'>EEEE</span><span style='color:{colour_id[territory_occupation[34][0]]}'>HHHHH</span><span style='color:{colour_id[territory_occupation[32][0]]}'>FFF</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[2][0]]}'>ddddddd</span><span style='color:{colour_id[territory_occupation[7][0]]}'>eeeee<span><span style='color:{colour_id[territory_occupation[8][0]]}'>fffff&nbsp;&nbsp;f</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[20][0]]}'>qq</span><span style='color:{colour_id[territory_occupation[16][0]]}'>rrrr</span><span style='color:{colour_id[territory_occupation[19][0]]}'>ttttt</span><span style='color:{colour_id[territory_occupation[27][0]]}'>GGGGGG</span><span style='color:{colour_id[territory_occupation[37][0]]}'>B</span><span style='color:{colour_id[territory_occupation[28][0]]}'>IIII</span><span style='color:{colour_id[territory_occupation[34][0]]}'>HHHHHHHHH&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[9][0]]}'>gggggggg</span><span style='color:{colour_id[territory_occupation[4][0]]}'>hhh</span>&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[8][0]]}'>fffff</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[20][0]]}'>qq</span><span style='color:{colour_id[territory_occupation[18][0]]}'>sssss</span>&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[19][0]]}'>ttt</span>&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[27][0]]}'>GGGGGG</span><span style='color:{colour_id[territory_occupation[28][0]]}'>IIIII</span><span style='color:{colour_id[territory_occupation[34][0]]}'>HHHHHHHH&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[9][0]]}'>gggggggg</span><span style='color:{colour_id[territory_occupation[4][0]]}'>hhhhhh</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[20][0]]}'>qqq</span>&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[18][0]]}'>ss&nbsp;&nbsp;ss</span>&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[19][0]]}'>tt</span>&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[27][0]]}'>GGGGGG</span><span style='color:{colour_id[territory_occupation[28][0]]}'>IIIIIIIIIII</span>&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[31][0]]}'>J&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[9][0]]}'>ggggggg</span><span style='color:{colour_id[territory_occupation[4][0]]}'>hhhhh</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[20][0]]}'>qq</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[18][0]]}'>s&nbsp;&nbsp;ss</span><span style='color:{colour_id[territory_occupation[33][0]]}'>AAAAAA</span><span style='color:{colour_id[territory_occupation[29][0]]}'>KKK</span><span style='color:{colour_id[territory_occupation[27][0]]}'>GG</span><span style='color:{colour_id[territory_occupation[28][0]]}'>IIIIIIIII&nbsp;&nbsp;I</span>&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[31][0]]}'>JJ&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[9][0]]}'>ggggg</span><span style='color:{colour_id[territory_occupation[4][0]]}'>hhhhhhh</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[25][0]]}'>uuuu</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[33][0]]}'>AAAAA</span><span style='color:{colour_id[territory_occupation[29][0]]}'>KKKKKK</span><span style='color:{colour_id[territory_occupation[28][0]]}'>IIIIIIII</span>&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[31][0]]}'>JJ&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[9][0]]}'>gggg</span><span style='color:{colour_id[territory_occupation[4][0]]}'>hhh&nbsp;&nbsp;&nbsp;&nbsp;h</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[25][0]]}'>uuuuuu</span><span style='color:{colour_id[territory_occupation[23][0]]}'>vvvv</span><span style='color:{colour_id[territory_occupation[33][0]]}'>AAA&nbsp;&nbsp;A</span><span style='color:{colour_id[territory_occupation[29][0]]}'>KKKKKKK</span><span style='color:{colour_id[territory_occupation[28][0]]}'>IIIIIII</span>&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[31][0]]}'>J&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[3][0]]}'>iii</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[25][0]]}'>uuuuuuu</span><span style='color:{colour_id[territory_occupation[23][0]]}'>vvvv</span>&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[33][0]]}'>AAAA</span>&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[29][0]]}'>KKKKKK</span><span style='color:{colour_id[territory_occupation[35][0]]}'>LLLL</span><span style='color:{colour_id[territory_occupation[28][0]]}'>II&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[3][0]]}'>iii</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[25][0]]}'>uuuuuuu</span><span style='color:{colour_id[territory_occupation[23][0]]}'>vvvvv</span>&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[33][0]]}'>AAA</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[29][0]]}'>KKKK</span>&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[35][0]]}'>LLLL&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[3][0]]}'>iiiii</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[25][0]]}'>uuuuuuuu</span><span style='color:{colour_id[territory_occupation[22][0]]}'>wwww</span>&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[33][0]]}'>AA</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[29][0]]}'>KK</span>&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[35][0]]}'>LLL&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[3][0]]}'>iiii&nbsp;&nbsp;</span><span style='color:{colour_id[territory_occupation[13][0]]}'>jj</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[25][0]]}'>uuuuuuuu</span><span style='color:{colour_id[territory_occupation[22][0]]}'>wwwww</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[29][0]]}'>K</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[35][0]]}'>L</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[40][0]]}'>M</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[13][0]]}'>jjjjj</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[25][0]]}'>uuuuuu</span><span style='color:{colour_id[territory_occupation[21][0]]}'>xx</span><span style='color:{colour_id[territory_occupation[22][0]]}'>wwwww</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[29][0]]}'>K</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[40][0]]}'>M&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;MM</span>&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[41][0]]}'>NN</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[12][0]]}'>k</span><span style='color:{colour_id[territory_occupation[13][0]]}'>jjj</span><span style='color:{colour_id[territory_occupation[11][0]]}'>mmmm</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[25][0]]}'>u</span><span style='color:{colour_id[territory_occupation[21][0]]}'>xx</span><span style='color:{colour_id[territory_occupation[22][0]]}'>www</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[40][0]]}'>MMMM</span>&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[41][0]]}'>NNNN&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[12][0]]}'>kk</span><span style='color:{colour_id[territory_occupation[11][0]]}'>mmmmmmmm</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[21][0]]}'>xxx</span><span style='color:{colour_id[territory_occupation[22][0]]}'>www</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[41][0]]}'>N&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[12][0]]}'>kkk</span><span style='color:{colour_id[territory_occupation[11][0]]}'>mmmmm</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[21][0]]}'>xxx</span><span style='color:{colour_id[territory_occupation[26][0]]}'>yyy</span>&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[24][0]]}'>zz</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[39][0]]}'>PPPP&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[10][0]]}'>l</span><span style='color:{colour_id[territory_occupation[12][0]]}'>kk</span><span style='color:{colour_id[territory_occupation[11][0]]}'>mmmm</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[26][0]]}'>yyyyy</span>&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[24][0]]}'>z</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[42][0]]}'>OO</span><span style='color:{colour_id[territory_occupation[39][0]]}'>PPPPP&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[10][0]]}'>lllll</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[26][0]]}'>yyyy</span>&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[24][0]]}'>z</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[42][0]]}'>OOOOO</span><span style='color:{colour_id[territory_occupation[39][0]]}'>PPPP&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[10][0]]}'>llll</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[26][0]]}'>yyy</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[42][0]]}'>OOOO</span><span style='color:{colour_id[territory_occupation[39][0]]}'>PPPP&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[10][0]]}'>lll</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[26][0]]}'>yy</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[42][0]]}'>OO</span>&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[39][0]]}'>PPPP&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:{colour_id[territory_occupation[10][0]]}'>l&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span>

""")


&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:red'>aaa</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:red'>bbbbb&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;b</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>cccccc</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>pp</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>t</span><span style='color:black'>B&nbsp;&nbsp;B&nbsp;&nbsp;B</span><span style='color:black'>CCCCC</span><span style='color:black'>DDDDDDDD</span><span style='color:black'>FFFF&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span>
&nbsp;&nbsp;<span style='color:red'>aaaaaaaa</span><span style='color:red'>bbbbbbbbbbbb</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>cccc</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>ppp</span><span style='color:black'>tt&nbsp;&nbsp;ttt</span><span style='color:black'>BBBBB</span><span style='color:black'>CCCCC</span><span style='color:black'>DDDDDDD</span><span style='color:black'>FFFFFFFFFFF&nbsp;&nbsp;</span>
&nbsp;&nbsp;<span style='color:red'>aaaaaaaa</span><span style='color:red'>bbbbbbbbbbbbb</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>ccc</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>nnn</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>pp&nbsp;&nbsp;p</span><span style='color:black'>tttttt</span><span style='color:black'>BBBBB</span><span style='color:black'>CCCCC</span><span style='color:black'>DDDDD</span><span style='color:black'>FFFFFFFFFFFF&nbsp;&nbsp;F</span>
&nbsp;&nbsp;<span style='color:red'>aaaaaaaaa</span><span style='color:blue'>dddddddd</span><span style='color:black'>ee</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>fff</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>c</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>o</span>&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>pp</span>&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>ttttttt</span><span style='color:black'>BBBBB</span><span style='color:black'>CCC</span><span style='color:black'>EEEEEEE</span><span style='color:black'>FFFFFFFFFF</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:red'>a&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;a</span><span style='color:blue'>dddddddd</span><span style='color:black'>eeee</span>&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>ffff</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>oo</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>p</span>&nbsp;&nbsp;<span style='color:black'>r</span><span style='color:black'>ttttttt</span><span style='color:black'>BBBBB</span><span style='color:black'>CC</span><span style='color:black'>EEEEEEEE</span><span style='color:black'>FFF&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;F</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:red'>a</span><span style='color:blue'>dddddddd</span><span style='color:black'>eeeee</span><span style='color:black'>ffffff</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>ooo</span>&nbsp;&nbsp;<span style='color:black'>rrrr</span><span style='color:black'>ttttt</span><span style='color:black'>GGGG</span><span style='color:black'>BBB</span><span style='color:black'>CC</span><span style='color:black'>EEEE</span><span style='color:black'>HHHHH</span><span style='color:black'>FFF</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:blue'>ddddddd</span><span style='color:black'>eeeee<span><span style='color:black'>fffff&nbsp;&nbsp;f</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>qq</span><span style='color:black'>rrrr</span><span style='color:black'>ttttt</span><span style='color:black'>GGGGGG</span><span style='color:black'>B</span><span style='color:black'>IIII</span><span style='color:black'>HHHHHHHHH&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>gggggggg</span><span style='color:red'>hhh</span>&nbsp;&nbsp;<span style='color:black'>fffff</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>qq</span><span style='color:black'>sssss</span>&nbsp;&nbsp;<span style='color:black'>ttt</span>&nbsp;&nbsp;<span style='color:black'>GGGGGG</span><span style='color:black'>IIIII</span><span style='color:black'>HHHHHHHH&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>gggggggg</span><span style='color:red'>hhhhhh</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>qqq</span>&nbsp;&nbsp;<span style='color:black'>ss&nbsp;&nbsp;ss</span>&nbsp;&nbsp;<span style='color:black'>tt</span>&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>GGGGGG</span><span style='color:black'>IIIIIIIIIII</span>&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>J&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>ggggggg</span><span style='color:red'>hhhhh</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>qq</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>s&nbsp;&nbsp;ss</span><span style='color:black'>AAAAAA</span><span style='color:black'>KKK</span><span style='color:black'>GG</span><span style='color:black'>IIIIIIIII&nbsp;&nbsp;I</span>&nbsp;&nbsp;<span style='color:black'>JJ&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>ggggg</span><span style='color:red'>hhhhhhh</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>uuuu</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>AAAAA</span><span style='color:black'>KKKKKK</span><span style='color:black'>IIIIIIII</span>&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>JJ&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>gggg</span><span style='color:red'>hhh&nbsp;&nbsp;&nbsp;&nbsp;h</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>uuuuuu</span><span style='color:black'>vvvv</span><span style='color:black'>AAA&nbsp;&nbsp;A</span><span style='color:black'>KKKKKKK</span><span style='color:black'>IIIIIII</span>&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>J&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:red'>iii</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>uuuuuuu</span><span style='color:black'>vvvv</span>&nbsp;&nbsp;<span style='color:black'>AAAA</span>&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>KKKKKK</span><span style='color:black'>LLLL</span><span style='color:black'>II&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:red'>iii</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>uuuuuuu</span><span style='color:black'>vvvvv</span>&nbsp;&nbsp;<span style='color:black'>AAA</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>KKKK</span>&nbsp;&nbsp;<span style='color:black'>LLLL&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:red'>iiiii</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>uuuuuuuu</span><span style='color:black'>wwww</span>&nbsp;&nbsp;<span style='color:black'>AA</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>KK</span>&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>LLL&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:red'>iiii&nbsp;&nbsp;</span><span style='color:black'>jj</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>uuuuuuuu</span><span style='color:black'>wwwww</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>K</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>L</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>M</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>jjjjj</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>uuuuuu</span><span style='color:black'>xx</span><span style='color:black'>wwwww</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>K</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>M&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;MM</span>&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>NN</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>k</span><span style='color:black'>jjj</span><span style='color:black'>mmmm</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>u</span><span style='color:black'>xx</span><span style='color:black'>www</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>MMMM</span>&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>NNNN&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>kk</span><span style='color:black'>mmmmmmmm</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>xxx</span><span style='color:black'>www</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>N&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>kkk</span><span style='color:black'>mmmmm</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>xxx</span><span style='color:black'>yyy</span>&nbsp;&nbsp;<span style='color:black'>zz</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>PPPP&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>l</span><span style='color:black'>kk</span><span style='color:black'>mmmm</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>yyyyy</span>&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>z</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>OO</span><span style='color:black'>PPPPP&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>lllll</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>yyyy</span>&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>z</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>OOOOO</span><span style='color:black'>PPPP&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>llll</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>yyy</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>OOOO</span><span style='color:black'>PPPP&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>lll</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>yy</span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>OO</span>&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>PPPP&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:black'>l&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span>



In [74]:
print(f"""
   aaa       //////             .....       pp         tm m m/////DDDDDDDDFFFF       
  a    a////   {territory_occupation[6][1]}  ////         . {territory_occupation[5][1]} .            pppptt  ttttttmmmm/////DDDDDDDFFFFFFFFFFF 
 a   {territory_occupation[1][1]}  a///////////            .  .    {territory_occupation[15][1]}      p {territory_occupation[17][1]}  ptttttttttmmmmm/////DDDDDFFFFFFFFFFFF F
 a     aa\\\\\\\\\\\\\\eee    fff       .         -    ppppttttttttttmmmmm///EEEEEEEFFFFFFFFFF    
  aaaa a\  {territory_occupation[2][1]}  \e   e  f   f              - {territory_occupation[14][1]}-    p  ..ttttttttmmmmm//EEEEEEEEFFF    F      
    aaa\\\\\    \e  {territory_occupation[7][1]} ef  {territory_occupation[8][1]}  f             ---   .......tttttGGGGmmm//EEEEHHHHHFFF          
          \\\\\\\\\\\\eeeef    f                c  cc.  {territory_occupation[16][1]}  .tttttGGGGGGmIIIIHHHHHHHHH           
          ggggggg// ffff                 cc{territory_occupation[20][1]}cc\\\\\\\\..ttt GGGGGGIIIIIHHHHHHHH           
          g     g/////                    cccc \\ {territory_occupation[18][1]} \\ tt  GGGGGGIIIIIIIIIII  J          
          g  {territory_occupation[9][1]} g/   /                    ccc  \\  \\\\  AAAAAAKKKGGIIIIIIIII I JJ          
           g   g/ {territory_occupation[4][1]}  /                               A    AKKKKKKIIIIIIII  JJ           
            ggg//    /                   ///////ddd    AAAA  AKKKKKKKIIIIIII  J            
              kk/  //                  /     /d {territory_occupation[23][1]} dd AA  A   KKKKKKLLLLII               
              k kk/                   /        //ddddd     AA A   KKKK LLLL                 
               k   k                  /    {territory_occupation[25][1]}   //wwww    A A     KK  LLL                  
                k {territory_occupation[3][1]} k/////            /       /w     w       K     L   M              
                  k/  {territory_occupation[13][1]}  /mmm           /     /xw  {territory_occupation[22][1]}  w       K   M   MM  NN          
                    ..///mm   mm         ////x xw   ww             MMMM  NNNN          
                  . {territory_occupation[12][1]} .m   {territory_occupation[11][1]}   m           x {territory_occupation[21][1]} xwwww                       N         
                   ..   .m   m             xxxx...    zz                PPPP          
                    \\\\...mmmm              ..   ..     {territory_occupation[24][1]}               OOPPPPP         
                     \\   \\\\                 . {territory_occupation[26][1]} .     z             OOOOOPPPP        
                     \\  {territory_occupation[10][1]} \\\\                . ..                    OOOOPPPP        
                      \\  \\                  ..                    OO  PPPP    
                       \\\\                   

""")


   aaa       //////             .....       pp         tm m m/////DDDDDDDDFFFF       
  a    a////   1  ////         . 0 .            pppptt  ttttttmmmm/////DDDDDDDFFFFFFFFFFF 
 a   4  a///////////            .  .    0      p 0  ptttttttttmmmmm/////DDDDDFFFFFFFFFFFF F
 a     aa\\\\\\\eee    fff       .         -    ppppttttttttttmmmmm///EEEEEEEFFFFFFFFFF    
  aaaa a\  1  \e   e  f   f              - 0-    p  ..ttttttttmmmmm//EEEEEEEEFFF    F      
    aaa\\\    \e  0 ef  0  f             ---   .......tttttGGGGmmm//EEEEHHHHHFFF          
          \\\\\\eeeef    f                c  cc.  0  .tttttGGGGGGmIIIIHHHHHHHHH           
          ggggggg// ffff                 cc0cc\\\\..ttt GGGGGGIIIIIHHHHHHHH           
          g     g/////                    cccc \ 0 \ tt  GGGGGGIIIIIIIIIII  J          
          g  0 g/   /                    ccc  \  \\  AAAAAAKKKGGIIIIIIIII I JJ          
           g   g/ 1  /                               A    AKKKKKKIIIIIIII  JJ           
           

In [53]:
print(f"""
  aa       bbbb b         cccccc          pp     tB B BCCCCCDDDDDDDDFFFF       
 aaaaaaabbbbbbbbbb        cccc           ppptt tttBBBBBCCCCCDDDDDDDFFFFFFFFFFF 
 aaaaaaabbbbbbbbbbb       ccc   nnn     pp pttttttBBBBBCCCCCDDDDDFFFFFFFFFFFF F
 aaaaaaaaddddddde   fff    c        o  pp  tttttttBBBBBCCCEEEEEEEFFFFFFFFFF    
  a     adddddddeee  fff           oo   p rtttttttBBBBBCCEEEEEEEEFFF    F      
        adddddddeeeefffff          ooo rrrrtttttGGGGBBBCCEEEEHHHHHFFF          
          ddddddeeeeffff f           qqrrrrtttttGGGGGGBIIIIHHHHHHHHH           
          ggggggghh ffff             qqsssss ttt GGGGGGIIIIIHHHHHHHH           
          ggggggghhhhh             qqq ss ss tt  GGGGGGIIIIIIIIIII  J          
           gggggghhhh              qq   s ssAAAAAAKKKGGIIIIIIIII I JJ          
           gggghhhhhh               uuuu     AAAAAKKKKKKIIIIIIII  JJ           
            ggghhh h               uuuuuuvvvvAAA AKKKKKKKIIIIIII  J            
              ii                  uuuuuuuvvvv AAAA  KKKKKKLLLLII               
              ii                  uuuuuuuvvvvv AAA   KKKK LLLL                 
               iii                uuuuuuuuwwww AA     KK  LLL                  
                  iiijj           uuuuuuuuwwwww       K     L   M              
                    jjjjj          uuuuuuxxwwwww       K   M   MM  NN          
                    kjjjmmmm            uxxwww              MMMM  NNNN          
                    kkmmmmmmmm          xxxwww                       N         
                     kkkmmmmm           xxxyyy zz                PPPP          
                      lkkmmmm           yyyyy  z               OOPPPPP         
                      lllll              yyyy  z              OOOOOPPPP        
                      llll               yyy                   OOOOPPPP        
                      lll                yy                    OO  PPPP        
""")


  aa       bbbb b         cccccc          pp     tB B BCCCCCDDDDDDDDFFFF       
 aaaaaaabbbbbbbbbb        cccc           ppptt tttBBBBBCCCCCDDDDDDDFFFFFFFFFFF 
 aaaaaaabbbbbbbbbbb       ccc   nnn     pp pttttttBBBBBCCCCCDDDDDFFFFFFFFFFFF F
 aaaaaaaaddddddde   fff    c        o  pp  tttttttBBBBBCCCEEEEEEEFFFFFFFFFF    
  a     adddddddeee  fff           oo   p rtttttttBBBBBCCEEEEEEEEFFF    F      
        adddddddeeeefffff          ooo rrrrtttttGGGGBBBCCEEEEHHHHHFFF          
          ddddddeeeeffff f           qqrrrrtttttGGGGGGBIIIIHHHHHHHHH           
          ggggggghh ffff             qqsssss ttt GGGGGGIIIIIHHHHHHHH           
          ggggggghhhhh             qqq ss ss tt  GGGGGGIIIIIIIIIII  J          
           gggggghhhh              qq   s ssAAAAAAKKKGGIIIIIIIII I JJ          
           gggghhhhhh               uuuu     AAAAAKKKKKKIIIIIIII  JJ           
            ggghhh h               uuuuuuvvvvAAA AKKKKKKKIIIIIII  J            
              ii                  uuuuu

![](img/risk.jpg)

# initial territory split
initial_card_shuffle=random.sample(range(1, 43), 42)
card_shuffle=random.sample(range(44), 44)

In [ ]:
ducky=Player(1)
sloth=Player(2)

In [ ]:
ducky.manoeuvre()

In [ ]:
ducky.attack()

In [ ]:
ducky.occupied_territories.keys()

In [ ]:
ducky.win_check()

In [ ]:
ducky.continent_check()

In [ ]:
{i: len(j) for i, j in continent_id.items()}

In [ ]:
test={
    1: 1,
    2: 1,
    3: 1,
    4: 1,
    5: 1,
    6: 1,
    7: 1,
    8: 1,
    9: 1,
    10: 1
}

In [ ]:
set(continent_id['North America']).issubset(test.keys())

In [ ]:
set(continent_id['North America'])

In [ ]:
{i: set(j).issubset(test.keys()) for i,j in continent_id.items()} #return dictionary of continent and whether player owns the whole of it